**Q1.** Write a `CREATE TABLE` statement in the cell below that creates a table named `MonthNames` containing two columns:

*  `MonthNumber`, a non-null integer
*  `MonthName`, a variable length character value whose length is at most **12** characters

Do not include a schema name for the `MonthNames` table.  This means it will be created in the `dbo` schema.  Objects in the `dbo` schema do not need to be preferenced with a schema name, as they are the default

In [6]:
use [jack.rutherford];
CREATE TABLE MonthNames(
    MonthNumber INT NOT NULL,
    MonthName   CHAR(12)
);

: Msg 2714, Level 16, State 6, Line 2
There is already an object named 'MonthNames' in the database.

Total execution time: 00:00:00.020

**Q2.** Write a **single** `INSERT` statement in the cell below which inserts 12 rows into the `MonthNames` table, one for each month, with:

- 1 =\> January
- 2 =\> February
- ...
- 12 =\> December

In [ ]:
use [jack.rutherford];
INSERT INTO MonthNames (MonthNumber, MonthName)
VALUES
    (1, 'January'),
    (2, 'February'),
    (3, 'March'),
    (4, 'April'),
    (5, 'May'),
    (6, 'June'),
    (7, 'July'),
    (8, 'August'),
    (9, 'September'),
    (10, 'October'),
    (11, 'November'),
    (12, 'December');

In [7]:
SELECT * from MonthNames

(12 rows affected)

Total execution time: 00:00:00.174

MonthNumber,MonthName
1,January
2,February
3,March
4,April
5,May
6,June
7,July
8,August
9,September
10,October


As we have seen in the past, Azure Data Studio does not consistently detect new tables until after a restart.  

Restart ADS before moving on.

**Q3.** Display a count of the number of movies made for each value of `PrimaryGenre` since the year 2000 (including 2000) for which both:

- at least 1000 movies that have an IMDB rating were made
- <span style="color: var(--vscode-foreground);">the average IMDB rating of the movies in that genre was at least 6</span>

Include the average of the IMDB ratings for the movies within a genre, and only show it to 2 decimal places.  Show the genre with the \*\*highest\*\* average IMDB rating first.

There are **6** rows. The results should look like this:

| Genre | Number of movies | Average IMDB Rating |
| --- | --- | --- |
| Documentary | 11532 | 7.24 |
| Biography | 2790 | 7.11 |
| ... | ... | ... |
| Crime | 2817 | 6.01 |

In [ ]:
USE [jack.rutherford];
SELECT PrimaryGenre AS Genre,
       COUNT(TitleID) AS [Number of Movies],
       ROUND(AVG(IMDBRating), 2) AS [Average IMDB Rating]
FROM imdb.Movies
WHERE YearReleased >= 2000
GROUP BY PrimaryGenre
HAVING COUNT(IMDBRating) >= 1000 AND AVG(IMDBRating) >= 6
ORDER BY [Average IMDB Rating] DESC;

**Q4.** For the year 2020 only, show the shortest, longest, and number of movies by primary genre.  For this one, be sure to include all genres, even if no movies were made in that year.  Do not include any movies that don't have a value for the `RunningTimeInMinutes` column.

Order the result by genre so that they look like this:

| Genre | Shortest | Longest | Number of movies |
| --- | --- | --- | --- |
| Action | 45 | 200 | 404 |
| Adult | NULL | NULL | 0
| ... | ... | ... |
| Western | 70 | 140 | 14 |

In [8]:
use [jack.rutherford];

WITH AllGenres AS (
    SELECT DISTINCT PrimaryGenre
    FROM imdb.Movies
    WHERE PrimaryGenre IS NOT NULL
)
SELECT AllGenres.PrimaryGenre,
       MIN(M.RunningTimeInMinutes) as Shortest,
       MAX(M.RunningTimeInMinutes) as Longest,
       COUNT(M.TitleID) as [Number of Movies]
FROM AllGenres
LEFT JOIN imdb.Movies AS M ON AllGenres.PrimaryGenre = M.PrimaryGenre 
    AND M.YearReleased = 2020 
    AND M.RunningTimeInMinutes IS NOT NULL
GROUP BY AllGenres.PrimaryGenre
ORDER BY AllGenres.PrimaryGenre ASC

(26 rows affected)

Total execution time: 00:00:00.150

PrimaryGenre,Shortest,Longest,Number of Movies
Action,45,200,404
Adult,NULL,NULL,0
Adventure,45,154,158
Animation,45,382,117
Biography,40,160,126
Comedy,45,354,773
Crime,45,242,156
Documentary,28,669,1252
Drama,30,1325,1131
Family,46,141,62


**Q5**.  Create a query that displays a table similar to this one

| Month | Year | Number of orders |
| --- | --- | --- |
| September | 2017 | 159 |
| October | 2017 | 143 |
| ... | ... | .. |
| January | 2018 | 179 |
| ... | ... | ... |
| March | 2018 | 9 |

The contents of the `Number of orders` column are the **number of orders placed** during the given month and year.  The results are sorted first by year, and then by month.  Be careful to sort the months in **chronological** rather than **alphabetical** order.

You can use the `month` and `year` functions to help you get the month and year associated with a date. You will want to use the `MonthNames` table you created earlier in your query. Since that table is in a different database, you will have to explicitly reference the database that it's in, like this:

```
select * 
from [stu.dent].dbo.MonthNames

```

In [5]:
use [SalesOrdersExample];
SELECT MonthNames.MonthName as [Month],
       YEAR(OrderDate) as [Year],
       COUNT(OrderNumber) as [Number of orders]
FROM Orders
JOIN [jack.rutherford].dbo.MonthNames ON MonthNames.MonthNumber = cast(format(OrderDate,'MM') as int)
GROUP BY MonthName, MONTH(OrderDate), YEAR(OrderDate)
ORDER BY [Year], MONTH(OrderDate)

(7 rows affected)

Total execution time: 00:00:00.048

Month,Year,Number of orders
September,2017,159
October,2017,143
November,2017,147
December,2017,145
January,2018,179
February,2018,162
March,2018,9


**Q6.** Repeat the previous question, except include a running total of the number of orders made in that year, like this:

| Month | Year | Monthly Orders | Yearly Orders to Date |
| --- | --- | --- | --- |
| September | 2017 | 159 | 159 |
| October | 2017 | 143 | 302 |
| November | 2017 | 147 | 449 |
| December | 2017 | 145 | 594 |
| January | 2018 | 179 | 179 |
| February | 2018 | 162 | 341 |
| March | 2018 | 9 | 350 |

In [4]:
use [SalesOrdersExample];
SELECT
    MonthNames.MonthName AS [Month],
    YEAR(OrderDate) AS [Year],
    COUNT(OrderNumber) AS [Number of orders],
    ROW_NUMBER() OVER (PARTITION BY YEAR(OrderDate) ORDER BY MONTH(OrderDate)) AS [Running Total]
  FROM Orders
  JOIN [jack.rutherford].dbo.MonthNames ON MonthNames.MonthNumber = CAST(FORMAT(OrderDate, 'MM') AS INT)
  GROUP BY MonthName, MONTH(OrderDate), YEAR(OrderDate)

(7 rows affected)

Total execution time: 00:00:00.045

Month,Year,Number of orders,Running Total
September,2017,159,1
October,2017,143,2
November,2017,147,3
December,2017,145,4
January,2018,179,1
February,2018,162,2
March,2018,9,3


In [1]:
use [SalesOrdersExample];
WITH MonthlyOrderCounts AS (
  SELECT
    MonthNames.MonthName AS [Month],
    YEAR(OrderDate) AS [Year],
    COUNT(OrderNumber) AS [Number of orders],
    ROW_NUMBER() OVER (PARTITION BY YEAR(OrderDate) ORDER BY MONTH(OrderDate)) AS RowNum
  FROM Orders
  JOIN [jack.rutherford].dbo.MonthNames ON MonthNames.MonthNumber = CAST(FORMAT(OrderDate, 'MM') AS INT)
  GROUP BY MonthName, MONTH(OrderDate), YEAR(OrderDate)
)

SELECT
  [Month],
  [Year],
  [Number of orders] as [Monthly Orders],
  SUM([Number of orders]) OVER (PARTITION BY [Year] ORDER BY RowNum) AS [Running Total]
FROM MonthlyOrderCounts
ORDER BY [Year], RowNum

(7 rows affected)

Total execution time: 00:00:01.012

Month,Year,Monthly Orders,Running Total
September,2017,159,159
October,2017,143,302
November,2017,147,449
December,2017,145,594
January,2018,179,179
February,2018,162,341
March,2018,9,350


**Q7****.** <span style="color: var(--vscode-foreground);"> In the </span> `BowlingLeagueExample` <span style="color: var(--vscode-foreground);"> database, show the following per match values for each team:</span>

- minimum team raw score
- minimum team handicap score
- average team raw score
- average team handicap score
- maximum team raw score
- maximum team handicap score

for any team whose **maximum team handicap score** was larger than its **average team handicap score** by at least 100 pins.  **Team raw score** is the total value of the `RawScore` values in the `Bowler_Scores` table per (`TeamID`, `MatchID`) combination. **Team handicap score** is computed similarly.

Order the results in descending order by the average team handicap score.

(3 rows, the first should be for the _Manatees_ with an average team handicap score of 2354)

In [ ]:
use [BowlingLeagueExample];
WITH bowlingScores AS(
    SELECT MatchID, TeamName, SUM(HandiCapScore) [Team HandicapScore], SUM(RawScore) [Team Raw Score] FROM Bowler_Scores
    JOIN Bowlers on Bowler_Scores.BowlerID = Bowlers.BowlerID
    JOIN Teams on Bowlers.TeamID = Teams.TeamID
    GROUP BY TeamName, MatchID
)
SELECT TeamName,
    MIN([Team Raw Score]) [Min Team Raw Score],
    MIN([Team HandicapScore]) [Min Team HandicapScore],
    AVG([Team Raw Score]) [Avg Team Raw Score],
    AVG([Team HandicapScore]) [Avg Team HandicapScore],
    MAX([Team Raw Score]) [Max Team Raw Score],
    MAX([Team HandicapScore]) [Max Team HandicapScore]
FROM bowlingScores
GROUP BY TeamName
HAVING MAX([Team HandicapScore]) - AVG([Team HandicapScore]) >= 100

**Q8**. Find recipes that don't contain any ingredients classified as _meat_, but do contain 2 or more _spice_ ingredients.  Ingredient classifications like _meat_ and _spice_ can be found in the `Ingredient_Classes` table.  You will also need to use the `Recipes`, `Ingredients`, and `Recipe_Ingredients` tables in your answer.

Display the _recipe title_, _recipe class description_, and the _total number of ingredients_ for each recipe. 

(4 rows)

In [1]:
use [RecipesExample];
SELECT R.RecipeTitle, RC.RecipeClassDescription, COUNT(I.IngredientID) as [Total Ingredients]
FROM Recipes AS R
JOIN Recipe_Ingredients AS RI ON RI.RecipeID = R.RecipeID
JOIN Ingredients AS I ON I.IngredientID = RI.IngredientID
JOIN Recipe_Classes AS RC ON RC.RecipeClassID = R.RecipeClassID
LEFT JOIN (
    SELECT RI.RecipeID, SUM(CASE WHEN IC.IngredientClassDescription = 'meat' THEN 1 ELSE 0 END) as MeatCount,
                               SUM(CASE WHEN IC.IngredientClassDescription = 'spice' THEN 1 ELSE 0 END) as SpiceCount
    FROM Recipe_Ingredients AS RI
    JOIN Ingredients AS I ON I.IngredientID = RI.IngredientID
    JOIN Ingredient_Classes AS IC ON IC.IngredientClassID = I.IngredientClassID
    GROUP BY RI.RecipeID
) as Counts ON R.RecipeID = Counts.RecipeID
WHERE Counts.MeatCount = 0 AND Counts.SpiceCount >= 2
GROUP BY R.RecipeTitle, RC.RecipeClassDescription;


(4 rows affected)

Total execution time: 00:00:00.075

RecipeTitle,RecipeClassDescription,Total Ingredients
Salsa Buena,Hors d'oeuvres,5
Fettuccini Alfredo,Main course,7
"Huachinango Veracruzana (Red Snapper, Veracruz style)",Main course,12
Salmon Filets in Parchment Paper,Main course,10


**Q9.** Write a query to display the total number of students enrolled in each `Category` of courses by semester. Show the results by semester, and sort the results within a semester so the category having the most students enrolled is shown **first**. If two categories have the same number of total students in a semester, show them alphabetically.

(18 rows, _English_ and _Math_ should be the first two courses for semester 1 with 18 total students enrolled.)

In [ ]:
use [SchoolSchedulingExample];
SELECT
    SemesterNumber,
    CategoryDescription,
    [Total Students]
FROM (
    SELECT
        SemesterNumber,
        CategoryDescription,
        COUNT(Students.StudentID) AS [Total Students],
        RANK() OVER (PARTITION BY SemesterNumber ORDER BY COUNT(Students.StudentID) DESC, CategoryDescription) as CategoryRank
    FROM Categories
    JOIN Subjects ON Subjects.CategoryID = Categories.CategoryID
    JOIN Classes ON Classes.SubjectID = Subjects.SubjectID
    JOIN Student_Schedules ON Student_Schedules.ClassID = Classes.ClassID
    JOIN Students ON Students.StudentID = Student_Schedules.StudentID
    GROUP BY SemesterNumber, CategoryDescription
) RankedCategories
ORDER BY SemesterNumber, CategoryRank;


**Q10.** Using the data from the previous question, show only those categories whose position between semesters **changed**. The results should be sorted so the category whose position changed **the most** should be shown first, and should look like this

| Category | S1 Position | S2 Position | Change in Position |
| --- | --- | --- | --- |
| History | 9 | 3 | Up 6 |
| Music | 4 | 9 | Down 5 |
| Computer Information Systems | 8 | 5 | Up 3 |
| Accounting | 3 | 4 | Down 2 |
| Art | 3 | 4 | Down 1 |

I used the [abs](https:\learn.microsoft.com\en-us\sql\t-sql\functions\abs-transact-sql?view=sql-server-2016) and [convert](https:\learn.microsoft.com\en-us\sql\t-sql\functions\cast-and-convert-transact-sql?view=sql-server-2016) functions in my result, as well as the `+` operator for string concatentation.

In [ ]:
USE SchoolSchedulingExample;

WITH StudentsInCategories AS (
    SELECT
        SemesterNumber,
        CategoryDescription,
        [Total Students],
        CategoryRank
    FROM (
        SELECT
            SemesterNumber,
            CategoryDescription,
            COUNT(Students.StudentID) AS [Total Students],
            RANK() OVER (PARTITION BY SemesterNumber ORDER BY COUNT(Students.StudentID) DESC, CategoryDescription) as CategoryRank
        FROM Categories
        JOIN Subjects ON Subjects.CategoryID = Categories.CategoryID
        JOIN Classes ON Classes.SubjectID = Subjects.SubjectID
        JOIN Student_Schedules ON Student_Schedules.ClassID = Classes.ClassID
        JOIN Students ON Students.StudentID = Student_Schedules.StudentID
        GROUP BY SemesterNumber, CategoryDescription
    ) RankedCategories
)

SELECT
    C1.CategoryDescription AS [Category],
    C1.CategoryRank AS [S1 Position],
    C2.CategoryRank AS [S2 Position],
    CASE
        WHEN C2.CategoryRank > C1.CategoryRank THEN 'Down ' + CONVERT(NVARCHAR(255), C2.CategoryRank - C1.CategoryRank)
        WHEN C2.CategoryRank < C1.CategoryRank THEN 'Up ' + CONVERT(NVARCHAR(255), C1.CategoryRank - C2.CategoryRank)
        ELSE 'No change'
    END AS PositionChange
FROM StudentsInCategories C1
JOIN StudentsInCategories C2 ON C1.CategoryDescription = C2.CategoryDescription
WHERE C1.SemesterNumber < C2.SemesterNumber
   AND C1.CategoryRank <> C2.CategoryRank
ORDER BY ABS(C2.CategoryRank - C1.CategoryRank) DESC;


## Handing it in

Click the "eraser" icon in the top right corner of the ADS window to clear the results from all cells in your notebook. Then go to Moodle and upload the notebook using the **SQL** **Homework 2** assignment in the "Assignments" section of the site.